In [32]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Membaca data
file_path = "nutrisi.csv"  
data = pd.read_csv(file_path)
print("Data sebelum normalisasi:") 
print(data)

Data sebelum normalisasi:
    Unnamed: 0                     Menu  Energy  Protein   Fat  Cholesterol  \
0            1                 mie ayam     590      4.8   0.7          0.0   
1            2                mie bakso     590      4.8   0.7          0.0   
2            3                mie basah     590      4.8   0.7          0.0   
3            4               mie kering    1360      9.6   6.4          0.0   
4            5               mie kering    1180      9.6   1.4          0.0   
5            6                 mie soun    1594      0.3   0.1          0.0   
6            7                    misoa    1297     10.6   1.5          0.0   
7            8              nasi goreng    1046      3.5  17.2         26.0   
8            9              nasi jagung     506      2.8   0.6          0.0   
9           10               nasi liwet     544      2.4   0.2          0.0   
10          11  nasi lunak and kangkung     423      2.4   0.2          0.0   
11          12            

In [33]:
# Fungsi Normalisasi dengan Min-Max Scaling
def normalize_min_max(data):
    return (data - np.min(data, axis=0)) / (np.max(data, axis=0) - np.min(data, axis=0))

# Salin dataset untuk normalisasi
data_normalized = data.copy()

# Daftar kolom yang akan dinormalisasi
columns_to_normalize = ['Energy', 'Protein', 'Fat', 'Cholesterol', 'Vitamin A', 'Vitamin E']

# Normalisasi semua kolom yang dipilih
data_normalized[columns_to_normalize] = normalize_min_max(data_normalized[columns_to_normalize])

# Menampilkan hasil setelah normalisasi
print(data_normalized.round(2))


    Unnamed: 0                     Menu  Energy  Protein   Fat  Cholesterol  \
0            1                 mie ayam    0.23     0.44  0.03         0.00   
1            2                mie bakso    0.23     0.44  0.03         0.00   
2            3                mie basah    0.23     0.44  0.03         0.00   
3            4               mie kering    0.79     0.90  0.28         0.00   
4            5               mie kering    0.66     0.90  0.06         0.00   
5            6                 mie soun    0.95     0.00  0.00         0.00   
6            7                    misoa    0.74     1.00  0.06         0.00   
7            8              nasi goreng    0.56     0.31  0.77         0.36   
8            9              nasi jagung    0.17     0.24  0.02         0.00   
9           10               nasi liwet    0.20     0.20  0.00         0.00   
10          11  nasi lunak and kangkung    0.11     0.20  0.00         0.00   
11          12               nasi putih    0.20     

In [34]:
# Inisialisasi centroid awal secara manual
K = 3
initial_centroids = np.array([
    [0.56, 0.31, 0.77, 0.36, 0.05, 1.00],  #C1
    [0.11, 0.20, 0.00, 0.00, 0.39, 1.00],
    [0.43, 0.40, 0.09, 1.00, 0.16, 0.00]  
])

# Fungsi Euclidean Distance - menghitung dua titik antara centroid dengan data
def euclidean_distance(a, b):
    return np.sqrt(np.sum((a - b) ** 2))

# Fungsi untuk menghitung SSE
def calculate_sse(clusters, centroids):
    sse = 0
    for i, cluster in enumerate(clusters):
        for point in cluster:
            sse += euclidean_distance(point, centroids[i])
    return sse

# Fungsi K-Means - penetuan klaster
def k_means(data, centroids, max_iters=10, tolerance=1e-6):
    num_iters = 0
    while True:
        num_iters += 1

        # Alokasikan data ke centroid terdekat
        clusters = [[] for _ in range(len(centroids))]
        for point in data:
            distances = [euclidean_distance(point, centroid) for centroid in centroids]
            closest_centroid = np.argmin(distances)
            clusters[closest_centroid].append(point)

        sse = calculate_sse(clusters, centroids)

        # Tampilkan kluster dan centroid
        print(f"\nIterasi {num_iters} - SSE = {sse:.2F}:")
        for i, cluster in enumerate(clusters):
            print(f"Kluster {i + 1} ({len(cluster)} titik):")
        
        # Hitung centroid baru
        new_centroids = []
        for cluster in clusters:
            if cluster:  # Jika cluster tidak kosong
                new_centroids.append(np.mean(cluster, axis=0))
            else:  # Jika cluster kosong, gunakan centroid lama
                new_centroids.append(centroids[len(new_centroids)])

        # Tampilkan pusat kluster setelah pembaruan
        print(f"Pusat kluster pada Iterasi {num_iters}:")
        for i, centroid in enumerate(new_centroids):
            print(f"Centroid {i + 1}: {centroid.round(2)}")
        
        # Periksa konvergensi
        if np.allclose(centroids, new_centroids, rtol=tolerance):
            break
        centroids = new_centroids

        # Berhenti jika sudah mencapai iterasi maksimum
        if num_iters >= max_iters:
            print("Algoritma berhenti karena mencapai iterasi maksimum.")
            break

    return new_centroids, clusters, num_iters

In [35]:
print("\nCentroid awal (dideklarasikan manual):")
for i, centroid in enumerate(initial_centroids):
    print(f"Centroid {i + 1}: {centroid.round(3)}")
# Menjalankan algoritma K-Means 
data_array = data_normalized[columns_to_normalize].to_numpy()  # Mengonversi ke array NumPy
final_centroids, final_clusters, num_iters = k_means(data_array, initial_centroids)


Centroid awal (dideklarasikan manual):
Centroid 1: [0.56 0.31 0.77 0.36 0.05 1.  ]
Centroid 2: [0.11 0.2  0.   0.   0.39 1.  ]
Centroid 3: [0.43 0.4  0.09 1.   0.16 0.  ]

Iterasi 1 - SSE = 29.00:
Kluster 1 (2 titik):
Kluster 2 (4 titik):
Kluster 3 (24 titik):
Pusat kluster pada Iterasi 1:
Centroid 1: [0.78 0.48 0.88 0.18 0.03 1.  ]
Centroid 2: [0.08 0.16 0.   0.   0.41 0.25]
Centroid 3: [0.48 0.48 0.08 0.06 0.01 0.  ]

Iterasi 2 - SSE = 13.57:
Kluster 1 (2 titik):
Kluster 2 (4 titik):
Kluster 3 (24 titik):
Pusat kluster pada Iterasi 2:
Centroid 1: [0.78 0.48 0.88 0.18 0.03 1.  ]
Centroid 2: [0.08 0.16 0.   0.   0.41 0.25]
Centroid 3: [0.48 0.48 0.08 0.06 0.01 0.  ]


In [36]:
# Mengonversi final_clusters menjadi label klaster untuk setiap data
data_labels = []
for cluster_index, cluster in enumerate(final_clusters):
    for _ in cluster:
        data_labels.append(cluster_index)
data_labels = np.array(data_labels)

# Menggabungkan semua kluster menjadi satu array untuk evaluasi
data_np = np.vstack(final_clusters)  # Menggabungkan semua kluster menjadi satu array

# Menghitung Silhouette Score, Calinski-Harabasz Index, dan Davies-Bouldin Index
sc = silhouette_score(data_np, data_labels)
ch_index = calinski_harabasz_score(data_np, data_labels) 
db_index = davies_bouldin_score(data_np, data_labels) 

# Menampilkan hasil evaluasi
print("\nHasil Evaluasi Klasterisasi")
print(f"Silhouette Score: {sc:.2f}")
print(f"Calinski-Harabasz Index: {ch_index:.2f}")
print(f"Davies-Bouldin Index: {db_index:.2f}")      


Hasil Evaluasi Klasterisasi
Silhouette Score: 0.31
Calinski-Harabasz Index: 9.43
Davies-Bouldin Index: 1.15
